<a href="https://colab.research.google.com/github/rowanclarke/manim-math220/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Initialise
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install requests==2.26.0
!pip install manim
!pip install IPython --upgrade

from manim import *
import numpy as np

_vec = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
_rand = [[1, 1, 2], [1, 2, 4], [-1, 0, 1]]
_vec2 = np.dot(_vec, np.transpose(_rand))

_symm = [[2, 1, 1], [1, 2, 1], [1, 1, 4]]

w, v, wD, wS, wDS, vS, vT, vTS = 0
def update(m):
    w, v = np.linalg.eig(m)
    wD = np.diag(w);
    def shorten(e):
        e3 = round(e, 2)
        e = round(e)
        if (e == e3):
            return e
        else:
            return e3
    wS = list(map(shorten, w))
    wDS = np.diag(wS);
    vS = list(map(lambda e: list(map(shorten, e)), v))
    vT = np.transpose(v)
    vTS = list(map(lambda e: list(map(shorten, e)), vT))

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Building dependency tree       
Reading state infor

Manim Community v0.17.1

TypeError: ignored

# Spectral Decomposition and Single Value Decomposition

by Rowan Clarke, Aditi Dhillon, Alessandro Curioni, Ethan Darbyshire, Jonathan Baccari

## Transformations

We can visualise matrices by thier transformations on the identity.

In [ ]:
#@title Visualise Matrix as a Transformation
%%manim -ql -v WARNING VectorTransformation

mat = _rand
vec = _vec
vec2 = _vec2

class VectorTransformation(ThreeDScene):
    def construct(self):
        ax = ThreeDAxes()
        colors = [RED, GREEN, BLUE]
        def vec2arr(v):
            return list(map(lambda e:
                    Arrow3D(
                        start=np.array([0, 0, 0]),
                        end=np.array(e),
                        resolution=8)
                    , v))
        arrs_before = vec2arr(vec)
        arrs_after = vec2arr(vec2)
        m = Matrix(mat).to_corner(UL)
        self.add_fixed_in_frame_mobjects(m)
        self.set_camera_orientation(phi=2*PI/5, theta=-PI/5)
        self.add(ax, *arrs_before)
        self.begin_3dillusion_camera_rotation()
        self.play(Create(m))
        self.play(m.animate.set_column_colors(*colors), *list(map(lambda arri: arri[1].animate.set_color(colors[arri[0]]), enumerate(arrs_before))))
        self.wait()
        self.play(*list(map(lambda arri: arri[1].animate.become(arrs_after[arri[0]]), enumerate(arrs_before))))
        self.wait(duration=3)
        

In [ ]:
#@title Transforming a Sphere
%%manim -ql -v WARNING SphereTransformation

mat = _rand

class SphereTransformation(ThreeDScene):
    def construct(self):
        ax = ThreeDAxes()
        m = Matrix(mat).to_corner(UL)
        self.add_fixed_in_frame_mobjects(m)
        self.set_camera_orientation(phi=2*PI/5, theta=PI/5)
        sphere = Sphere()
        self.add(ax, sphere)
        self.begin_3dillusion_camera_rotation()
        self.play(ApplyMatrix(mat, sphere))
        self.wait(duration=3)

In [ ]:
#@title Diagonal Matrices are Enlargements 
mat = [[5, 0, 0], [0, 3, 0], [0, 0, 1]]

%manim -ql -v WARNING SphereTransformation

In [ ]:
#@title Orthogonal Matrices are Rotations 
mat = [[1, 0, 0], [0, 0, -1], [0, 1, 0]]

%manim -ql -v WARNING SphereTransformation

## Spectral Decomposition

> Q: Why are orthogonal and diagonal matrices interesting? (Hint: Spectral Decomposition)

Because for all real symmetric matrices $A$, there exists an orthoganal matrix $P$ and diagonal matrix $D$ such that $$A=PDP^\top$$

So all real symmetric matrices can be represented by a rotation, enlargement, and rotation (where the last rotation is the inverse of the first.)

In [ ]:
#@title Transformation of a Real Symmetric Matrix

mat = _symm

%manim -ql -v WARNING SphereTransformation

In [ ]:
#@title Spectral Decomposition
%%manim -ql -v WARNING SpectralDecomposition

mat = _symm

class SpectralDecomposition(Scene):
    def construct(self):
        update(mat)
        def mat2str(matrix):
            return r"\left[\begin{matrix}" + r"\\".join(map(lambda r: "&".join(map(lambda e: str(e), r)), matrix)) + r"\end{matrix}\right]"

        m = MathTex(r"A=" + mat2str(mat)).move_to([-5,2.5,0])
        t = MathTex(r"c_A(x)=" + "".join(map(lambda e: "(" + str(e) + " - " + "x)", wS))).next_to(m, buff=1)
        d = MathTex(r"D=" + mat2str(wDS)).next_to(m, direction=DOWN, buff=0.5)
        p = MathTex(r"P=" + mat2str(vS)).next_to(d, buff=1)
        pT = MathTex(r"P^\top=" + mat2str(vTS)).next_to(p, direction=DOWN, buff=0.5)
        
        self.add(m)
        self.wait()
        self.play(Create(t))
        self.play(Transform(t.copy(), d), Create(p))
        self.play(Transform(p.copy(), pT))
        self.wait(duration=3)

UsageError: Cell magic `%%manim` not found.


In [ ]:
#@title Transformation of Eigenvectors

%%manim -ql -v WARNING SphereTransformationVectors

mat = _symm
update(mat)
vec = _vec
vec2 = _vec2

class SphereTransformationVectors(ThreeDScene):
    def construct(self):
        ax = ThreeDAxes()
        colors = [RED, GREEN, BLUE]
        arrs_before = list(map(lambda e:
                    Arrow3D(
                        start=np.array([0, 0, 0]),
                        end=np.array(e),
                        resolution=8)
                    , vT))
        arrs_after = list(map(lambda ie:
                    Arrow3D(
                        color=colors[ie[0]],
                        start=np.array([0, 0, 0]),
                        end=np.array(ie[1]),
                        resolution=8)
                    , np.transpose(np.dot(mat, v)) ))
        m = Matrix(mat).to_corner(UL)
        self.add_fixed_in_frame_mobjects(m)
        self.set_camera_orientation(phi=2*PI/5, theta=PI/5)
        sphere = Sphere(fill_opacity=0.4)
        self.add(ax, sphere)
        self.begin_3dillusion_camera_rotation()
        self.play(ApplyMatrix(mat, sphere))
        #self.wait(duration=3)

UsageError: Cell magic `%%manim` not found.


In [ ]:
#@title Transformations of Spectral Decomposition
%%manim -ql -v WARNING SphereSpectralDecomposition

mat = _symm

class SphereSpectralDecomposition(ThreeDScene):
    def construct(self):
        update(mat)
        ax = ThreeDAxes()
        p = Matrix(vS).to_corner(UL)
        d = Matrix(wDS).to_corner(UL)
        pT = Matrix(vTS).to_corner(UL)
        self.set_camera_orientation(phi=2*PI/5, theta=-PI/5)
        sphere = Sphere()
        self.add(ax, sphere)
        self.begin_3dillusion_camera_rotation()
        self.add_fixed_in_frame_mobjects(pT)
        self.play(FadeIn(pT), ApplyMatrix(vT, sphere))
        self.wait()
        self.add_fixed_in_frame_mobjects(d)
        self.play(FadeOut(pT), FadeIn(d), ApplyMatrix(wD, sphere))
        self.wait()
        self.add_fixed_in_frame_mobjects(p)
        self.play(FadeOut(d), FadeIn(p), ApplyMatrix(v, sphere))
        self.wait(duration=3)

UsageError: Cell magic `%%manim` not found.


## Single Value Decomposition

So that's cool! However...

> Pick any matrix. It's likely to be non-symmetric.

... it only works on real symmetric matrices.

If only there was a way to decompose any real matrix into a rotation, enlargement, and rotation.

There is!

We need to find an orthogonal matrix $V$ such that $AV$ has orthogonal vectors, or rather $$AV=U\Sigma$$ where $\Sigma$ is diagonal and $U$ is orthogonal.

Then we have $$A=U\Sigma V^\top$$

## A Return to Spectral Decomposition

Consider $AA^\top$.

$AA^\top = (U\Sigma V^\top)(U\Sigma V^\top)^\top=(U\Sigma V^\top)(V\Sigma^\top U^\top)$

So $AA^\top = U\Sigma\Sigma^\top U^\top$ because $V^{-1}=V^\top$

> Notice what happened to the dimension.

Since $AA^\top$ is symmetric, we can use spectral decomposition!

$U$ is the orthonormal basis of eigenvectors and $\Sigma\Sigma^\top$ is the diagonal matrix of eigenvalues of $AA^\top$. 

We can do the same for $A^\top A$ and find that $V$ is the orthonormal basis of eigenvectors and $\Sigma^\top\Sigma$ is the diagonal matrix of eigenvalues of $A^\top A$.

> So what is $\Sigma$? We know it's diagonal, right?

But $\Sigma$ is rectangular. Since the non-zero eigenvalues of $A^\top A$ and $A A^\top$ are the same, $\Sigma$ is the **rectangular** diagonal matrix root of non-zero eigenvalues.

## But why?

If $A$ represents a massive dataset, then the orthogonal basis vectors in $U$ corresponding to the largest eigenvalues is the weighting of each dimension which have the most profound effect on the data. This process is called Principle Component Analysis. 